In [4]:
import pandas as pd

diabetic = pd.read_csv("New_train_set.csv")
diabetic = diabetic.drop(diabetic.columns[0],axis = 1)

In [7]:
from sklearn.model_selection import KFold
import numpy as np

from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score

def back_wrapper(clf,X_raw,y,k):
    X = X_raw.copy()
    Wrap_accuracy_old = 0
    Wrap_accuracy_new = 0.00001
    n = X.shape[1]
    #generate an empty dataframe to save features
    Wrap_filt = X.copy()
    
    kf = KFold(n_splits=k, random_state=42, shuffle=False)
    sm = SMOTE(random_state = 42, ratio = 1)
    
    while(Wrap_accuracy_old <  Wrap_accuracy_new):
        Wrap_accuracy_old = Wrap_accuracy_new
        avg_accuracies_list = []

        for i in range(Wrap_filt.shape[1]):
            accur_list = []
            for train_index, test_index in kf.split(X):                
                
                X_res, y_res = sm.fit_sample(Wrap_filt.drop(Wrap_filt.columns[i],axis=1).iloc[train_index,:], y.iloc[train_index])
                clf.fit(X_res, y_res)
                y_pred = clf.predict(Wrap_filt.drop(Wrap_filt.columns[i],axis=1).iloc[test_index,:])
                
                score = f1_score(y.iloc[test_index],y_pred,average = "weighted")
                accur_list.append(score)
            avg_accuracies_list.append(np.mean(accur_list))
            
        # renew the accuracy
        Wrap_accuracy_new = np.sort(avg_accuracies_list)[-1]                 
        print(n,"selection is done")
        
        if Wrap_accuracy_old <  Wrap_accuracy_new:
            # add the highest accuracy feature to the selected dataframe
            Wrap_filt = Wrap_filt.drop(Wrap_filt.columns[np.argsort(avg_accuracies_list)[-1]])          
            n = n - 1            
        else:
            break

    return Wrap_accuracy_old,n,Wrap_filt

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

clf1 = GaussianNB()
clf2 = Perceptron(random_state=42, n_jobs=-1)
clf3 = RandomForestClassifier(n_estimators='warn',n_jobs=-1)
clf4 = LinearSVC()
clf5 = KNeighborsClassifier(n_neighbors=5)
clf6 = LogisticRegression()

In [ ]:
logit_acc, logit_num_feature, logit_features = back_wrapper(clf6,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5)

In [ ]:
logit_features.to_csv("Logit_Back_Smote.csv",index=False)
logit_acc

In [ ]:
NB_acc, NB_num_feature, NB_features = back_wrapper(clf1,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5)

In [ ]:
NB_features.to_csv("NB_Back_Smote.csv",index=False)
NB_acc

In [ ]:
Per_acc, Per_num_feature, Per_features = back_wrapper(clf2,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5)

In [ ]:
Per_features.to_csv("Per_Back_Smote.csv",index=False)
Per_acc

In [ ]:
RF_acc, RF_num_feature, RF_features = back_wrapper(clf3,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5)

In [ ]:
RF_features.to_csv("RF_Back_Smote.csv",index=False)
RF_acc

In [ ]:
SVM_acc, SVM_num_feature, SVM_features = back_wrapper(clf4,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5)

In [ ]:
SVM_features.to_csv("SVM_Back_Smote.csv",index=False)
SVM_acc

In [ ]:
knn_acc, knn_num_feature, knn_features = back_wrapper(clf5,diabetic.iloc[:,:-1],diabetic.iloc[:,-1],5)

In [ ]:
knn_features.to_csv("knn_Back_Smote.csv",index=False)
knn_acc

In [ ]:
modelname = ['NB',"Perceptron","RF","SVM","KNN","Logit"]
summary_acc = pd.DataFrame({"accuracy":[NB_acc,Per_acc,RF_acc,SVM_acc,knn_acc,logit_acc]},index=modelname)
summary_acc.to_csv("Accuracy_Back_Smote.csv")